In [23]:
#导入需要的库
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pylab as plt
from sklearn.datasets import make_classification

## make_classification参数解释

**n_samples**：默认100,生成样本数量<br>
**n_features**:默认20,总的特征数量,是从有信息的数据点，冗余数据点，重复数据点，和特征点-有信息的点-冗余的点-重复点中随机选择的。<br>
**n_informative**:<br>
**n_redundant**:冗余特征数<br>
**n_repeated**:<br>
**n_classes**:输出类别数<br>
**n_clusters_per_class**:<br>
**weights**:floats列表 or None，默认none。权重<br>
**flip_y**:<br>
**class_sep**:<br>
**hypercube**:<br>
**shift**:<br>
**scale**:<br>
**shuffle**:<br>
**random_state**:<br>
如果是int，random_state是随机数发生器使用的种子; <br>
如果RandomState实例，random_state是随机数生成器; <br>
如果没有，则随机数生成器是np.random使用的RandomState实例<br>

In [10]:
# 生成数据
X,y = make_classification(n_samples = 1000, # 样本数
                         n_features = 50, # 特征数
                         n_clusters_per_class = 1,
                         n_informative = 15,
                         random_state = 10)

In [11]:
print(X.shape)
print(y.shape)

(1000, 50)
(1000,)


In [12]:
# 所有参数都采用默认值，查看分类情况
rf0 = RandomForestClassifier(oob_score=True,random_state=10)
rf0.fit(X,y)
print(rf0.oob_score_)
print("准确率：%f" % rf0.oob_score_)

0.818
准确率：0.818000


d:\python\envs\jupyterpy36\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
d:\python\envs\jupyterpy36\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [14]:
# 对外层的bagging框架进行调优，即对n_estimators调优，其他参数仍然不变
# 1-101，步长10，十折交叉验证
param_test1 = {"n_estimators":range(1,101,10)}
gsearch1  = GridSearchCV(estimator = RandomForestClassifier(),
                      param_grid = param_test1,
                      scoring = "roc_auc",
                       cv = 10)
gsearch1.fit(X,y)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': range(1, 101, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [19]:
print(gsearch1.grid_scores_)
print("*"*30)
print(gsearch1.best_params_)
print("*"*30)
print("最佳评分：%f" % gsearch1.best_score_)

[mean: 0.72990, std: 0.05181, params: {'n_estimators': 1}, mean: 0.95332, std: 0.01877, params: {'n_estimators': 11}, mean: 0.96958, std: 0.00879, params: {'n_estimators': 21}, mean: 0.96846, std: 0.01212, params: {'n_estimators': 31}, mean: 0.97379, std: 0.00809, params: {'n_estimators': 41}, mean: 0.97730, std: 0.01004, params: {'n_estimators': 51}, mean: 0.97471, std: 0.01002, params: {'n_estimators': 61}, mean: 0.97753, std: 0.01011, params: {'n_estimators': 71}, mean: 0.97572, std: 0.00900, params: {'n_estimators': 81}, mean: 0.97678, std: 0.01185, params: {'n_estimators': 91}]
******************************
{'n_estimators': 71}
******************************
最佳评分：0.977535


d:\python\envs\jupyterpy36\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


### 最佳子决策树个数为71，准确率为97.75%

In [20]:
# 再来优化max_features最大特征数,1-11,步长1，十折
param_test2 = {"max_features":range(1,11,1)}
gsearch2 = GridSearchCV(estimator=RandomForestClassifier(n_estimators = 71,random_state = 10),
                       param_grid = param_test2,
                       scoring = "roc_auc",
                       cv = 10)
gsearch2.fit(X,y)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=71, n_jobs=1,
            oob_score=False, random_state=10, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_features': range(1, 11)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='roc_auc', verbose=0)

In [22]:
print(gsearch2.grid_scores_)
print("*"*30)
print(gsearch2.best_params_)
print("*"*30)
print("准确率：%f" % gsearch2.best_score_)

[mean: 0.93793, std: 0.01414, params: {'max_features': 1}, mean: 0.96973, std: 0.01120, params: {'max_features': 2}, mean: 0.96913, std: 0.01035, params: {'max_features': 3}, mean: 0.97448, std: 0.01074, params: {'max_features': 4}, mean: 0.97481, std: 0.01332, params: {'max_features': 5}, mean: 0.97250, std: 0.01241, params: {'max_features': 6}, mean: 0.97466, std: 0.01348, params: {'max_features': 7}, mean: 0.97401, std: 0.01226, params: {'max_features': 8}, mean: 0.97726, std: 0.00769, params: {'max_features': 9}, mean: 0.97843, std: 0.01199, params: {'max_features': 10}]
******************************
{'max_features': 10}
******************************
准确率：0.978434


d:\python\envs\jupyterpy36\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [24]:
# 所有参数都采用默认值，查看分类情况
rf0 = RandomForestRegressor(oob_score=True,random_state=10)
rf0.fit(X,y)
print(rf0.oob_score_)
print("准确率：%f" % rf0.oob_score_)

0.5528666043690711
准确率：0.552867


d:\python\envs\jupyterpy36\lib\site-packages\sklearn\ensemble\forest.py:724: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


In [25]:
# 对外层的bagging框架进行调优，即对n_estimators调优，其他参数仍然不变
# 1-101，步长10，十折交叉验证
param_test1 = {"n_estimators":range(1,101,10)}
gsearch1  = GridSearchCV(estimator = RandomForestRegressor(),
                      param_grid = param_test1,
                      scoring = "roc_auc",
                       cv = 10)
gsearch1.fit(X,y)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': range(1, 101, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [27]:
# 所有参数都采用默认值，查看分类情况
rf0 = ExtraTreesClassifier(oob_score=True,random_state=10,bootstrap=True)
rf0.fit(X,y)
print(rf0.oob_score_)
print("准确率：%f" % rf0.oob_score_)

0.757
准确率：0.757000


d:\python\envs\jupyterpy36\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
d:\python\envs\jupyterpy36\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [28]:
# 对外层的bagging框架进行调优，即对n_estimators调优，其他参数仍然不变
# 1-101，步长10，十折交叉验证
param_test1 = {"n_estimators":range(1,101,10)}
gsearch1  = GridSearchCV(estimator = ExtraTreesClassifier(),
                      param_grid = param_test1,
                      scoring = "roc_auc",
                       cv = 10)
gsearch1.fit(X,y)

GridSearchCV(cv=10, error_score='raise',
       estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': range(1, 101, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [29]:
print(gsearch1.grid_scores_)
print("*"*30)
print(gsearch1.best_params_)
print("*"*30)
print("最佳评分：%f" % gsearch1.best_score_)

[mean: 0.71382, std: 0.04969, params: {'n_estimators': 1}, mean: 0.94939, std: 0.01787, params: {'n_estimators': 11}, mean: 0.96464, std: 0.01070, params: {'n_estimators': 21}, mean: 0.97004, std: 0.01177, params: {'n_estimators': 31}, mean: 0.97528, std: 0.01199, params: {'n_estimators': 41}, mean: 0.97586, std: 0.01135, params: {'n_estimators': 51}, mean: 0.97704, std: 0.01232, params: {'n_estimators': 61}, mean: 0.97782, std: 0.00934, params: {'n_estimators': 71}, mean: 0.97961, std: 0.00905, params: {'n_estimators': 81}, mean: 0.97745, std: 0.01084, params: {'n_estimators': 91}]
******************************
{'n_estimators': 81}
******************************
最佳评分：0.979614


d:\python\envs\jupyterpy36\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
